In [2]:
import pandas as pd
import numpy as np
import csv

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.utils import shuffle

import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve, ShuffleSplit


In [3]:
addpoly = True
plot_lc = 0

print('----- Loading the data -----')
train_dataset = pd.read_csv('train.csv')
test_dataset = pd.read_csv('test.csv')

print('train dataset {} and test dataset {}'.format(str(train_dataset.shape), str(test_dataset.shape)))
train_dataset.head()

----- Loading the data -----
train dataset (891, 12) and test dataset (418, 11)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
if train_dataset.PassengerId.nunique() == train_dataset.shape[0] :
    print('ID is unique')
else:
    print('oops')

ID is unique


In [5]:
if len(np.intersect1d(train_dataset.PassengerId.values, test_dataset.PassengerId.values)) == 0:
    print('Training and test are unique')
else:
    print('oops')

Training and test are unique


In [6]:
datasethasnan = False
if (train_dataset.count().min() == train_dataset.shape[0] and test_dataset.count().min() == test_dataset.shape[0]):
    print('No missing/NaN values')
else:
    datasethasnan = True
    print('Found NaN values')

Found NaN values


In [7]:
print('<-----Train dataset column type information------>')
print(train_dataset.dtypes.reset_index().columns)

<-----Train dataset column type information------>
Index(['index', 0], dtype='object')


In [8]:
if (datasethasnan == True):
    nas = pd.concat([train_dataset.isnull().sum(), test_dataset.isnull().sum()], axis=1, keys=['Train Dataset', 'Test Dataset']) 
    print(nas)
    print('NaN in datasets: ')
    
    print(nas[nas.sum(axis=1) > 0])

             Train Dataset  Test Dataset
Age                    177          86.0
Cabin                  687         327.0
Embarked                 2           0.0
Fare                     0           1.0
Name                     0           0.0
Parch                    0           0.0
PassengerId              0           0.0
Pclass                   0           0.0
Sex                      0           0.0
SibSp                    0           0.0
Survived                 0           NaN
Ticket                   0           0.0
NaN in datasets: 
          Train Dataset  Test Dataset
Age                 177          86.0
Cabin               687         327.0
Embarked              2           0.0
Fare                  0           1.0


In [10]:
train_dataset.isnull().sum()


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [11]:
# Class vs Survived
print(train_dataset[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False))

   Pclass  Survived
0       1  0.629630
1       2  0.472826
2       3  0.242363


In [13]:
print('<----- Start data cleaning ----->')

<----- Start data cleaning ----->


In [14]:
train_random_ages = np.random.randint(train_dataset["Age"].mean()-train_dataset["Age"].std(),
                                     test_dataset["Age"].mean()-test_dataset["Age"].std(),
                                     size = train_dataset["Age"].isnull().sum())

In [15]:
print(train_random_ages)

[15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15 15
 15 15]


In [20]:
test_dataset["Age"].mean()-test_dataset["Age"].std()

16.091381125821364

In [21]:
train_dataset["Age"].mean()-train_dataset["Age"].std()

15.172620314724776